In [46]:
import pandas as pd
import numpy as np
import os
import sys
import requests
import matplotlib.pyplot as plt

from numpy import *

data_path = r'../data'
data_file = os.path.join(data_path, 'train.csv')
out_file = os.path.join(data_path, 'train2.csv')

df = pd.read_csv(data_file, encoding='utf-8')

In [52]:
"""
    major,
    cast_popularity,
    vote_avg,
    genre|major,
    us|major,
    cast_popularity|genre
    budget|major, genre
    movie_popularity|genre, us, cast_popularity, vote_avg_community, vote_avg_critics
    vote_count_community|movie_popularity
    vote_count_critics|movie_popularity
    revenue|movie_popularity
"""
groups = [
#     df.groupby('major'),
#     df.groupby('vote_average_binned'),
#     df.groupby('cast_popularity_binned'),
#     df.groupby(['major', macro_genre']),
#     df.groupby(['major', 'macro_genre']),
#     df.groupby(['major', 'us']),
#     df.groupby(['macro_genre', 'cast_popularity_binned']),
#     df.groupby(['major', 'cast_popularity_binned']),
    df.groupby(['major', 'macro_genre', 'budget_binned'])
]

# we don't have this, we want to make inference about it
#moviepop_groupby = df.groupby(['macro_genre', 'us', 'cast_popularity_category', 'vote_average_category'])

# compute the list of cpts for each group
for g in groups:
    group_count = g.size()
    names = group_count.index.names
    
#     print(group_count.unstack(fill_value=0).unstack(fill_value=0).unstack(fill_value=0).stack().stack().stack())
    
    # see https://stackoverflow.com/questions/42854801/including-missing-combinations-of-values-in-a-pandas-groupby-aggregation
    # we need to unstack each and every level to account for 0-count subgroups
    for lvl in range(1, len(names)):
        group_count = group_count.unstack(fill_value=0)
        
    for lvl in range(1, len(names)):
        group_count = group_count.stack()
        
    if(len(names) > 1):
        # group by all but the last column (the one we're conditioning on)
        levels = list(range(0, len(names)-1))
        conditional = group_count.groupby(level=levels).apply(lambda subg: subg/subg.sum())
#         joint = group_count/len(df)
        filename = '%s-%s.csv' % (names[-1], ','.join(names[0:-1]))
    else:
        conditional = group_count/group_count.sum()
#         joint = conditional
        filename = '%s.csv' % names[0]
        
    # Save cpt to csv files. One file per cpt
    conditional[isnan(conditional)] = 0
    conditional.to_csv(os.path.join(data_path, 'cpt', filename), header=True, encoding='utf-8')
        
#     print('joint:\n')
#     print(joint)
    print('conditional:\n')
    print(conditional)
    print('\n')

conditional:

major  macro_genre  budget_binned
0      action       1st              0.919890
                    2nd              0.078729
                    3rd              0.001381
       dark         1st              0.998127
                    2nd              0.001873
                    3rd              0.000000
       drama        1st              0.992958
                    2nd              0.007042
                    3rd              0.000000
       light        1st              0.984642
                    2nd              0.015358
                    3rd              0.000000
       other        1st              1.000000
                    2nd              0.000000
                    3rd              0.000000
1      action       1st              0.784499
                    2nd              0.204159
                    3rd              0.011342
       dark         1st              0.979675
                    2nd              0.020325
                    3rd         